In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

import shared_utils
import branca
from utils import *

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis2 as rt
import importlib

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [23]:
%%sql -o rt_coverage

# get all vehicle positions on selected dates, for the feed with itp id 170, and url number 0
SELECT calitp_itp_id, calitp_url_number, MIN(vehicle_timestamp) as min_timestamp, MAX(vehicle_timestamp) as max_timestamp
FROM `cal-itp-data-infra.gtfs_rt.vehicle_positions`
GROUP BY calitp_itp_id, calitp_url_number
# WHERE _FILE_NAME='gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-11-30_290_0.parquet' ##utc, must bracket 2 days for 1 day pacific
#     OR _FILE_NAME='gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-01_290_0.parquet'
ORDER BY max_timestamp DESC

,calitp_itp_id,calitp_url_number,min_timestamp,max_timestamp
0,61,1,1640723732,1642031994
1,110,1,1640723745,1642031994
2,183,0,1420070460,1642031990
3,314,1,1640817264,1642031989
4,380,0,1638643253,1642031989
...,...,...,...,...
59,349,0,1642013696,1642031848
60,294,0,1625804796,1640817236
61,290,0,1625804799,1640817217
62,167,0,1625804880,1640723712


In [25]:
## set system time
os.environ['TZ'] = 'America/Los_Angeles'
time.tzset()
time.tzname

('PST', 'PDT')

In [26]:
rt_coverage.max_timestamp = rt_coverage.max_timestamp.apply(convert_ts)
rt_coverage.min_timestamp = rt_coverage.min_timestamp.apply(convert_ts)

In [42]:
rt_coverage

,calitp_itp_id,calitp_url_number,min_timestamp,max_timestamp
0,61,1,2021-12-28 12:35:32,2022-01-12 15:59:54
1,110,1,2021-12-28 12:35:45,2022-01-12 15:59:54
2,183,0,2014-12-31 16:01:00,2022-01-12 15:59:50
3,314,1,2021-12-29 14:34:24,2022-01-12 15:59:49
4,380,0,2021-12-04 10:40:53,2022-01-12 15:59:49
...,...,...,...,...
59,349,0,2022-01-12 10:54:56,2022-01-12 15:57:28
60,294,0,2021-07-08 21:26:36,2021-12-29 14:33:56
61,290,0,2021-07-08 21:26:39,2021-12-29 14:33:37
62,167,0,2021-07-08 21:28:00,2021-12-28 12:35:12


In [43]:
rt_coverage >> inner_join(_, tbl.gtfs_schedule.agency() >> collect(), on = ['calitp_itp_id']) >> filter(_.calitp_itp_id == 290)

,calitp_itp_id,calitp_url_number_x,min_timestamp,max_timestamp,calitp_url_number_y,agency_id,agency_name,agency_url,agency_timezone,agency_lang,agency_phone,agency_fare_url,agency_email,calitp_extracted_at
123,290,1,2021-12-29 14:34:05,2022-01-12 15:59:23,1,SM,SamTrans,http://www.samtrans.com,America/Los_Angeles,en,800-660-4287,None,None,2022-01-05
124,290,1,2021-12-29 14:34:05,2022-01-12 15:59:23,0,samtrans-ca-us,SamTrans,http://www.samtrans.com,America/Los_Angeles,en,800-660-4287,None,None,2022-01-11
125,290,0,2021-07-08 21:26:39,2021-12-29 14:33:37,1,SM,SamTrans,http://www.samtrans.com,America/Los_Angeles,en,800-660-4287,None,None,2022-01-05
126,290,0,2021-07-08 21:26:39,2021-12-29 14:33:37,0,samtrans-ca-us,SamTrans,http://www.samtrans.com,America/Los_Angeles,en,800-660-4287,None,None,2022-01-11


In [27]:
rt_coverage >> inner_join(_, tbl.gtfs_schedule.agency() >> collect(), on = ['calitp_itp_id']) >> filter(_.calitp_itp_id == 182)

,calitp_itp_id,calitp_url_number_x,min_timestamp,max_timestamp,calitp_url_number_y,agency_id,agency_name,agency_url,agency_timezone,agency_lang,agency_phone,agency_fare_url,agency_email,calitp_extracted_at
95,182,1,2022-01-12 10:53:39,2022-01-12 15:59:43,1,LACMTA_Rail,Metro - Los Angeles,https://www.metro.net,America/Los_Angeles,en,(323) 466-3876,None,None,2022-01-12
96,182,1,2022-01-12 10:53:39,2022-01-12 15:59:43,0,LACMTA,Metro - Los Angeles,https://www.metro.net,America/Los_Angeles,en,(323)466-3876,None,None,2021-12-18
97,182,0,2022-01-12 10:45:32,2022-01-12 15:59:40,1,LACMTA_Rail,Metro - Los Angeles,https://www.metro.net,America/Los_Angeles,en,(323) 466-3876,None,None,2022-01-12
98,182,0,2022-01-12 10:45:32,2022-01-12 15:59:40,0,LACMTA,Metro - Los Angeles,https://www.metro.net,America/Los_Angeles,en,(323)466-3876,None,None,2021-12-18


In [22]:
%%sql
SELECT * FROM `cal-itp-data-infra.gtfs_rt.vehicle_positions`
LIMIT 5

,calitp_itp_id,calitp_url_number,entity_id,vehicle_id,vehicle_trip_id,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp
0,106,0,vehicle_2620,2620,14_Trip7_SP_COVID,1625883981,37.827232,-122.268880,1625883994
1,106,0,vehicle_2632,2632,7_Trip7_H_COVID,1625883981,37.829826,-122.267690,1625883994
2,106,0,vehicle_2634,2634,8_Trip7_SP_COVID,1625883981,37.832977,-122.293480,1625883994
3,106,0,vehicle_2635,2635,6_Trip7_H_COVID,1625883981,37.828460,-122.267204,1625883994
4,106,0,vehicle_3023,3023,12_Trip7_SP_COVID,1625883981,37.837040,-122.302780,1625883994


In [28]:
%%sql -o metro0_positions

# get all vehicle positions on selected dates, for the feed with itp id 170, and url number 0
SELECT *
FROM `cal-itp-data-infra.gtfs_rt.vehicle_positions`
WHERE _FILE_NAME='gs://gtfs-data/rt-processed/vehicle_positions/vp_2022-01-12_182_0.parquet' ##utc, must bracket 2 days for 1 day pacific
    # OR _FILE_NAME='gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-01_290_0.parquet'
ORDER BY header_timestamp

,calitp_itp_id,calitp_url_number,entity_id,vehicle_id,vehicle_trip_id,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp
0,182,0,5817,5817,10105002581045-DEC21,1642013714,34.080147,-118.376580,1642013729
1,182,0,5810,5810,10053002331050-DEC21,1642013671,34.054066,-118.250320,1642013729
2,182,0,5811,5811,10051003301041-DEC21,1642013569,34.047920,-118.257150,1642013729
3,182,0,5813,5813,None,1642013715,34.028564,-118.252525,1642013729
4,182,0,5814,5814,10055002841003-DEC21,1642013716,34.033695,-118.262460,1642013729
...,...,...,...,...,...,...,...,...,...
514894,182,0,5800,5800,10014005021606-DEC21,1642031974,34.035442,-118.368860,1642031988
514895,182,0,5801,5801,10002011261537-DEC21,1642031979,34.090210,-118.392490,1642031988
514896,182,0,5802,5802,10060001601558-DEC21,1642031974,33.925970,-118.210090,1642031988
514897,182,0,5803,5803,10212002141540-DEC21,1642031976,33.994614,-118.355760,1642031988


In [29]:
%%sql -o metro1_positions

# get all vehicle positions on selected dates, for the feed with itp id 170, and url number 0
SELECT *
FROM `cal-itp-data-infra.gtfs_rt.vehicle_positions`
WHERE _FILE_NAME='gs://gtfs-data/rt-processed/vehicle_positions/vp_2022-01-12_182_1.parquet' ##utc, must bracket 2 days for 1 day pacific
    # OR _FILE_NAME='gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-01_290_0.parquet'
ORDER BY header_timestamp

,calitp_itp_id,calitp_url_number,entity_id,vehicle_id,vehicle_trip_id,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp
0,182,1,1035-1047-1068,1035-1047-1068,54949456,1642013730,34.047234,-118.26005,1642013769
1,182,1,1057,1057,54922444,1642013762,34.047770,-118.22785,1642013769
2,182,1,1020-1028,1020-1028,54949453,1642013752,34.025080,-118.35799,1642013769
3,182,1,537-538-547-548-579-580,537-538-547-548-579-580,55035306,1642013745,34.164757,-118.37452,1642013769
4,182,1,1026-1064-1069,1026-1064-1069,54949454,1642013762,34.018290,-118.30410,1642013769
...,...,...,...,...,...,...,...,...,...
19744,182,1,549-550-563-564-601-602,549-550-563-564-601-602,55035174,1642031983,34.101720,-118.31097,1642031989
19745,182,1,1075-1088-1136,1075-1088-1136,54949672,1642031947,33.812210,-118.19219,1642031989
19746,182,1,1023-1054-1063,1023-1054-1063,54949768,1642031977,34.040997,-118.26587,1642031989
19747,182,1,567-568-583-584,567-568-583-584,55035251,1642031983,34.061680,-118.30957,1642031989


In [37]:
(metro1_positions
 >> distinct(_.vehicle_trip_id, _.vehicle_timestamp, _keep_all=True)
 >> group_by(_.vehicle_trip_id)
 >> summarize(n = _.vehicle_timestamp.size, trip_time = _.vehicle_timestamp.max() - _.vehicle_timestamp.min())
 >> mutate(update_interval = _.trip_time / _.n, trip_minutes = _.trip_time / 60)
 >> mutate(mean_update_interval = _.update_interval.mean())
)

,vehicle_trip_id,n,trip_time,update_interval,trip_minutes,mean_update_interval
0,54922026,41,2675,65.243902,44.583333,58.210082
1,54922031,31,1780,57.419355,29.666667,58.210082
2,54922032,55,3251,59.109091,54.183333,58.210082
3,54922033,51,2999,58.803922,49.983333,58.210082
4,54922035,42,2416,57.523810,40.266667,58.210082
...,...,...,...,...,...,...
422,55035406,18,979,54.388889,16.316667,58.210082
423,55035407,12,1028,85.666667,17.133333,58.210082
424,55035408,30,1782,59.400000,29.700000,58.210082
425,55035409,26,1433,55.115385,23.883333,58.210082


In [41]:
(metro0_positions
 >> distinct(_.vehicle_trip_id, _.vehicle_timestamp, _keep_all=True)
 >> group_by(_.vehicle_trip_id)
 >> summarize(n = _.vehicle_timestamp.size, trip_time = _.vehicle_timestamp.max() - _.vehicle_timestamp.min())
 >> mutate(update_interval = _.trip_time / _.n, trip_minutes = _.trip_time / 60)
 >> mutate(median_update_interval = _.update_interval.median())
)

,vehicle_trip_id,n,trip_time,update_interval,trip_minutes,median_update_interval
0,10002011231518-DEC21,50,2465,49.300000,41.083333,51.744673
1,10002011240911-DEC21,3,60,20.000000,1.000000,51.744673
2,10002011240930-DEC21,33,1800,54.545455,30.000000,51.744673
3,10002011240940-DEC21,36,1885,52.361111,31.416667,51.744673
4,10002011240949-DEC21,43,2620,60.930233,43.666667,51.744673
...,...,...,...,...,...,...
4583,10910002121411-DEC21,53,3333,62.886792,55.550000,51.744673
4584,10910002121431-DEC21,155,7401,47.748387,123.350000,51.744673
4585,10910002121451-DEC21,109,5260,48.256881,87.666667,51.744673
4586,10910002121511-DEC21,89,4220,47.415730,70.333333,51.744673
